In [1]:
import io

import tensorflow as tf
import numpy as np
from tensorflow import data
import csv
tf.enable_eager_execution()

/home/asr/tensorflow-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/asr/tensorflow-cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


=== Installing SpaCy and the last version of model **it_model0** contained in _packages_ folder. 

In [6]:

#!pip install --user spacy==2.1.4
!pip install --user packages/it_model0-0.0.1/dist/*.tar.gz

Processing ./packages/it_model0-0.0.1/dist/it_model0-0.0.1.tar.gz
  Created wheel for it-model0: filename=it_model0-0.0.1-cp35-none-any.whl size=11758457 sha256=65dd9b0eec9be268ee648f61e34e3105963c7b1c0c8f3f8ab1832f931d019c0c
  Stored in directory: /home/asr/.cache/pip/wheels/91/d8/59/e1925eccff69a71964d31e692e08bac8a53cda757664e1c7b8
Successfully built it-model0
  Found existing installation: it-model0 0.0.1
    Uninstalling it-model0-0.0.1:
      Successfully uninstalled it-model0-0.0.1


In [2]:
!pip list

Package                       Version               
----------------------------- ----------------------
apturl                        0.5.2                 
attrs                         19.3.0                
backcall                      0.1.0                 
beautifulsoup4                4.4.1                 
bleach                        3.1.0                 
blinker                       1.3                   
blis                          0.2.4                 
Brlapi                        0.6.4                 
certifi                       2019.9.11             
chardet                       3.0.4                 
checkbox-support              0.22                  
command-not-found             0.3                   
cryptography                  1.2.3                 
cymem                         2.0.2                 
decorator                     4.4.0                 
defer                         1.0.6                 
defusedxml                    0.6.0           

=== Loading module

In [4]:
import spacy
nlp = spacy.load("it_model0")
!python3 -m spacy info it_model0


======================== Info about model 'it_model0' ========================

vectors         {'name': 'spacy_pretrained_vectors', 'keys': 0, 'vectors': 0, 'width': 0}
version         0.0.1                         
pipeline        ['tagger', 'parser', 'ner']   
spacy_version   >=2.1.4                       
name            model0                        
source          /home/asr/.local/lib/python3.5/site-packages/it_model0
lang            it                            



In [73]:
print("ADP : {}".format(spacy.explain("ADP")))
print("ADV : {}".format(spacy.explain("ADV")))
print("VERB : {}".format(spacy.explain("VERB")))
print("PROPN : {}".format(spacy.explain("PROPN")))
print("NUM : {}".format(spacy.explain("NUM")))
print("NOUN : {}".format(spacy.explain("NOUN")))
print("CCONJ : {}".format(spacy.explain("CCONJ")))
print("DET : {}".format(spacy.explain("DET")))
print("VERB_CLIT : {}".format(spacy.explain("VERB_CLIT")))

ADP : adposition
ADV : adverb
VERB : verb
PROPN : proper noun
NUM : numeral
NOUN : noun
CCONJ : coordinating conjunction
DET : determiner
VERB_CLIT : None


=== Punctual Test

In [5]:
doc = nlp("che colore è lo scarlatto")
for token in doc:
    print(token.text, token.lemma_, token.tag_)

che che DET
colore colorire NOUN
è essere VERB
lo il DET
scarlatto scarlatto NOUN


In [75]:
doc = nlp("dove è andato luca")
for token in doc:
    print(token.text, token.lemma_, token.tag_, token.dep_)

dove dove ADV advmod
è essere AUX aux
andato andare VERB ROOT
luca lucere PROPN nsubj


In [448]:
def pos_sentence(sentence, POS = 'TAG'):
    
    # POS can be 'TAG' or 'TAG_LEMMA'
    
    # decode to strin type if not yet
    if isinstance(sentence, (bytes, bytearray)):
        sentence = sentence.decode()
        
    doc = nlp(str(sentence.rstrip()))

    text = [token.text for token in doc]
    lemma = [token.lemma_ for token in doc]
    tag = [token.tag_ for token in doc]
    
    PROPN_idxs = [i for i, value in enumerate(tag) if value == 'PROPN'] 
    
    if POS == 'TAG':
        # Only tag
        text_pos = ['PROPN' if i in PROPN_idxs else text[i] for i in range(len(text))]
    else:
        # Tag + Lemmatization
        text_pos = ['PROPN' if i in PROPN_idxs else lemma[i] for i in range(len(text))]
        
    text_pos = ' '.join(text_pos)
    
    return text_pos
 


pos_sentence(b'dove sono andati luca e sara', POS = 'TAG_LEMMA')

'dove essere andato PROPN e PROPN'

In [449]:
pos_sentence('dove sono andati luca e sara', POS = 'TAG_LEMMA')

'dove essere andato PROPN e PROPN'

In [450]:
pos_sentence(b'dove sono andati luca e sara', POS = 'TAG')

'dove sono andati PROPN e PROPN'

In [395]:

pos_sentence('dove sono andati luca e sara', POS = 'TAG_LEMMA')

'dove essere andato PROPN e PROPN'

In [436]:
pos_sentence(b'dove \xc3\xa8 andato luca', POS = 'TAG')



'dove è andato PROPN'

=== File Test

In [437]:
f = io.open("tests/ticket_test.txt", mode="r", encoding="utf-8")
strings = f.readlines() # it returns a simple strings list containing the whole text
f.close()

In [438]:
for string in strings:
    print("{}".format(string))
    print(pos_sentence(string, POS = 'TAG_LEMMA'))
    print('---------------------------------------')

selma sei un maschio o una femmina

PROPN essere un maschiare o una femmina
---------------------------------------
meglio apple o siri

meglio PROPN o siri
---------------------------------------
secondo te sofia è di milano

secondare te PROPN essere di PROPN
---------------------------------------
la mia fidanzata è susy

la mio fidanzato essere susy
---------------------------------------
lorenzo è furbo

PROPN essere furbo
---------------------------------------
secondo te giorgio è qui vicino

secondare te PROPN essere qui vicino
---------------------------------------
l iphone è migliore del samsung

l iphone essere buono del PROPN
---------------------------------------
a che ora è la gara di superbike

a che orare essere la gara di PROPN
---------------------------------------
addiziona una mela e una mela 

addizionare una mela e una mela
---------------------------------------
quanta ram ho

quanto PROPN avere
---------------------------------------
quanta ram ha il mio tele

chi avere costruire monumento al cadere
---------------------------------------
quali contatti ho aggiornato di recente

quale contatto avere aggiornare di recere
---------------------------------------
quanto spazio libero ho sul telefono

quanto spaziare liberare avere sul telefonare
---------------------------------------
quanta memoria libera ho sul telefono

quanto memoria libero avere sul telefonare
---------------------------------------
quando sono i mondiali di calcio

quando essere il mondiale di calciare
---------------------------------------
sarà soleggiato a milano oggi

essere soleggiare a PROPN oggi
---------------------------------------
sarà soleggiato a milano domani

essere soleggiare a PROPN domani
---------------------------------------
fuori fa freddo o caldo oggi

fuori fare freddare o caldo oggi
---------------------------------------
in che provincia è abbiategrasso

in che provincia essere abbiategrasso
---------------------------------------
in che provincia

registrare che il mio pesare essere cinquanticinque chilo
---------------------------------------
quali album ho nella galleria 

quale album avere nella galleria
---------------------------------------
manderesti a papà un email con scritto ciao usando gmail

PROPN a papà un email con scrivere ciao usare gmail
---------------------------------------
manderesti a mamma un email con scritto ciao usando gmail

PROPN a mamma un email con scrivere ciao usare gmail
---------------------------------------
ho sonnolenza

avere sonnolenza
---------------------------------------
ho le carie

avere la carie
---------------------------------------
che percentuale di batteria ho

che percentuale di batteria avere
---------------------------------------
segui sara su instagram

seguire sara su PROPN
---------------------------------------
il rum è alcolico

il rum essere alcolico
---------------------------------------
il film demons and angels è basato su una storia vera

il film PROPN and PROPN e

In [721]:
import re
def _parse_and_postag(tsv_row, pos = 'TAG', placeholder = "tannutuva"):

    columns = tf.decode_csv(tsv_row, record_defaults=[['NA'], ['NA']], field_delim='\t')
    
    if pos is not None: 
        postag_col = tf.py_func(lambda x: pos_sentence(x, POS = pos), [columns[0]], tf.string, stateful=False)
    else:
        postag_col = columns[0]
        
    if placeholder is not None:
        postag_col = tf.regex_replace(postag_col, r"\be\b", placeholder)
        postag_col = tf.regex_replace(postag_col, r"\Bè\B", placeholder)
    else:
        postag_col = postag_col if pos is not None else columns[0]
    
    features = {'sentence': postag_col, 'class': columns[1]}
    target = features.pop('class')

    return features, target

In [722]:
TRAIN_DATA_PATH='/home/asr/prj_SlotTagger/dnn_train/datasets/tmp/test.tsv'

In [723]:
! cat '/home/asr/prj_SlotTagger/dnn_train/datasets/tmp/test.tsv'

dove è andato luca e perchè	1
io e giulia saremo a milano	0
quanto è costato il treno per roma	1
come è andato il weekend	1
tu giulia e laura sarete a piacenza domani	0


In [724]:
file_names = tf.matching_files(TRAIN_DATA_PATH)

dataset = data.TextLineDataset(filenames=file_names)
dataset = dataset.skip(0)


In [725]:
for line in dataset.take(5):
    print(line)

tf.Tensor(b'dove \xc3\xa8 andato luca e perch\xc3\xa8\t1', shape=(), dtype=string)
tf.Tensor(b'io e giulia saremo a milano\t0', shape=(), dtype=string)
tf.Tensor(b'quanto \xc3\xa8 costato il treno per roma\t1', shape=(), dtype=string)
tf.Tensor(b'come \xc3\xa8 andato il weekend\t1', shape=(), dtype=string)
tf.Tensor(b'tu giulia e laura sarete a piacenza domani\t0', shape=(), dtype=string)


In [726]:
dataset = dataset.map(lambda tsv_row: _parse_and_postag(tsv_row, pos = 'TAG', placeholder=None))

In [727]:
for line in dataset.take(5):
    print(line)

({'sentence': <tf.Tensor: id=3192, shape=(), dtype=string, numpy=b'dove \xc3\xa8 andato PROPN e perch\xc3\xa8'>}, <tf.Tensor: id=3193, shape=(), dtype=string, numpy=b'1'>)
({'sentence': <tf.Tensor: id=3196, shape=(), dtype=string, numpy=b'io e PROPN saremo a PROPN'>}, <tf.Tensor: id=3197, shape=(), dtype=string, numpy=b'0'>)
({'sentence': <tf.Tensor: id=3200, shape=(), dtype=string, numpy=b'quanto \xc3\xa8 costato il treno per PROPN'>}, <tf.Tensor: id=3201, shape=(), dtype=string, numpy=b'1'>)
({'sentence': <tf.Tensor: id=3204, shape=(), dtype=string, numpy=b'come \xc3\xa8 andato il weekend'>}, <tf.Tensor: id=3205, shape=(), dtype=string, numpy=b'1'>)
({'sentence': <tf.Tensor: id=3208, shape=(), dtype=string, numpy=b'tu giulia e PROPN sarete a piacenza domani'>}, <tf.Tensor: id=3209, shape=(), dtype=string, numpy=b'0'>)


In [430]:
iterator = dataset.make_one_shot_iterator()
features, target = iterator.get_next()
features

{'sentence': <tf.Tensor: id=1330, shape=(), dtype=string, numpy=b'dove \xc3\xa8 andato PROPN'>}